In [ ]:
from sklearn import svm, preprocessing
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
#from statsmodels.api import datasets
from sklearn import datasets ## Get dataset from sklearn
import sklearn.model_selection as ms
import sklearn.metrics as sklm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.random as nr
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import sklearn.decomposition as skde
import math
import seaborn as sns
from sklearn.model_selection import GridSearchCV

%matplotlib inline

Code below is based on:
https://scikit-learn.org/stable/modules/ensemble.html#voting-classifier

In [ ]:
bikesData = pd.read_csv('BikesProcessed.csv')
Labels = bikesData['BikeBuyer']
bikesData.drop(bikesData.columns[0], axis=1, inplace=True)
bikesData.drop(['BikeBuyer'], axis=1, inplace=True)
Features = np.array(bikesData)
print(Features.shape)
bikesData.head()

In [ ]:
# Randomly sample cases to create independent training and test data

nr.seed(1115)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 5000)
X_train = Features[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
X_test = Features[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
tree = DecisionTreeClassifier(max_depth = i, random_state=0)
tree.fit(X_train, y_train)
print("accuracy on training set: %f" % tree.score(X_train, y_train))
print("accuracy on test set: %f" % tree.score(X_test, y_test))

In [ ]:
svc = SVC()
svc.fit(X_train, y_train)
print("accuracy on training set: %f" % svc.score(X_train, y_train))
print("accuracy on test set: %f" % svc.score(X_test, y_test))

In [ ]:
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
print("accuracy on training set: %f" % mlp.score(X_train, y_train))
print("accuracy on test set: %f" % mlp.score(X_test, y_test))

In [ ]:
nb = BernoulliNB() 
nb.fit(X_train, y_train)
print("accuracy on training set: %f" % mlp.score(X_train, y_train))
print("accuracy on test set: %f" % mlp.score(X_test, y_test))

In [ ]:
vote_hard = VotingClassifier(estimators=[('tree', tree), ('svc', svc), ('mlp', mlp), ('nb', nb)], voting='hard')
vote_hard.fit(X_train, y_train)
print("accuracy on training set: %f" % vote_hard.score(X_train, y_train))
print("accuracy on test set: %f" % vote_hard.score(X_test, y_test))

In [ ]:
for clf, label in zip([tree, svc, mlp, nb, vote_hard], ['Decision Tree', 'SVC', 'MLP', 'Naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
vote_soft = VotingClassifier(estimators=[('tree', tree), ('svc', svc), ('mlp', mlp), ('nb', nb)], voting='soft')
vote_soft.fit(X_train, y_train)
print("accuracy on training set: %f" % vote_soft.score(X_train, y_train))
print("accuracy on test set: %f" % vote_soft.score(X_test, y_test))

In [ ]:
params = {'lr__C': [1.0, 100.0], 'rf__n_estimators': [20, 200]}
grid = GridSearchCV(estimator=vote_soft, param_grid=params, cv=5)
grid = grid.fit(Features, Labels)